In [ ]:
pip install gdown

In [ ]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1--W7LMimW3QF7x00CMxAUuUYcgDgSoFa'

output = 'file.npy'

gdown.download(url, output, quiet=False)

In [ ]:
import numpy as np

In [ ]:
with open('file.npy', 'rb') as f:
    X_train = np.load(f)
    X_val = np.load(f)
    Y_train = np.load(f)
    Y_val = np.load(f)
X_train.shape

In [ ]:
def get_category(level):
    categories = ["Healthy","Mild", "Moderate", "Severe", "Proliferate"]
    return categories[level]

In [ ]:
#checking data

from matplotlib import pyplot as plt
plt.imshow(X_train[7], interpolation='nearest')
plt.title(get_category(int(Y_train[7].sum(axis=0)) - 1))
plt.show()

In [ ]:
import json
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
%matplotlib inline

In [ ]:
def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  # set range for random zoom
        # set mode for filling points outside the input boundaries
        fill_mode='constant',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,# randomly flip images
        rotation_range=360,
        rescale=1./255)
batch_size = 16
data_generator = create_datagen().flow(X_train, Y_train, batch_size=batch_size)

In [ ]:
from sklearn.metrics import accuracy_score
class Metrics(Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        

        X_val, y_val = self.validation_data[:2]
        y_val = y_val.sum(axis=1) - 1
        
        y_pred = self.model.predict(X_val) > 0.5
        y_pred = y_pred.astype(int).sum(axis=1) - 1
        
        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        print(f"Actual val_accuracy: {accuracy_score(y_val, y_pred)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved... Saving model!")
            model.save_weights('model.h5')
            model_json = model.to_json()
            with open('model.json', "w") as json_file:
                json_file.write(model_json)
            json_file.close()

        return

In [ ]:
IMG_SIZE = 256
from tensorflow.keras.applications import Xception

exception = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE,IMG_SIZE,3)
)

def build_model():
    model = Sequential()
    model.add(exception)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='sigmoid'))
    model.compile(
      loss='binary_crossentropy',#to check why we use it in binary classif#our class contains this severity or n?
      optimizer=Adam(lr=0.00005),
      metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,show_layer_names=True)


In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model, legend=True)

In [ ]:
kappa_metrics = Metrics((X_val,Y_val))
stopping=EarlyStopping(monitor='val_loss',patience=5, min_delta=0.005)
history = model.fit_generator(
    data_generator,
    steps_per_epoch=X_train.shape[0] // batch_size,
    validation_data=(X_val,Y_val),
    epochs=12,
    callbacks=[stopping,kappa_metrics])

In [ ]:
model.load_weights('model.h5')
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
val_kappas = kappa_metrics.val_kappas

epochs_range = range(len(loss))

plt.figure(figsize = (16, 16))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
#plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
#plt.plot(epochs_range, val_kappas, label = 'Validation Kappa')
plt.legend(loc = 'lower right')
plt.title('Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
y_pred = model.predict(X_val) > 0.5#get the probabilities of multiclasses
print(y_pred[:5])

In [ ]:
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
print(y_pred[:5])

In [ ]:
predictions = y_pred.reshape(1,-1)[0]
y_val =Y_val.astype(int).sum(axis=1) - 1#retrieve index 0 based
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, predictions, target_names=["C 0", "C 1", "C 2", "C 3", "C 4"]))

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
loss_object = BinaryCrossentropy()
import tensorflow as tf
def generate_image_adversary(model, image, label, eps=2 / 255.0):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        pred = model(image)
       # print(label, pred)
        #label = label.astype(int).sum(axis=0) - 1
        #pred = pred.astype(int).sum(axis=0) - 1
        label = tf.convert_to_tensor(label)# to have same form as the pred object
        label = tf.reshape(label, (1, 5))
        
        #print(label, pred)
        loss = loss_object(label, pred)
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)
        adversary = (image + (signedGrad * eps)).numpy()
    return adversary

In [ ]:
import numpy as np
def generate_adversarial_batch(model, total, images, labels, dims,eps=0.01):
    (h, w, c) = dims
    while True:
        perturbImages = []
        perturbLabels = []
        idxs = np.random.choice(range(0, len(images)), size=total,replace=False)
        for i in idxs:
            image = images[i]
            label = labels[i]
            adversary = generate_image_adversary(model,image.reshape(1, h, w, c), label, eps=eps)
            perturbImages.append(adversary.reshape(h, w, c))
            perturbLabels.append(label)
        yield (np.array(perturbImages), np.array(perturbLabels))

In [ ]:
y_pred = model.predict(X_val) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =Y_val.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Normal Accuracy : {acc}")

In [ ]:
print("generating adversarial examples with FGSM...")
(advX, advY) = next(generate_adversarial_batch(model, len(X_val),X_val, Y_val, (256,256, 3), eps=0.01))
# re-evaluate the model on the adversarial images


In [ ]:
y_pred = model.predict(advX) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =advY.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Adv Accuracy : {acc}")

In [ ]:
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
print("fine-tuning network on adversarial examples...")
model.fit(advX, advY,batch_size=16,epochs=5,verbose=1)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(advX[0], interpolation='nearest')
plt.title("Adversarial Image of stage:" + get_category(int(advY[1].sum(axis=0)) - 1))
plt.show()

In [ ]:
y_pred = model.predict(X_val) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =Y_val.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Normal Accuracy after fine-tuning: {acc}")

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
y_pred = model.predict(advX) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =advY.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Adv Accuracy after fine-tuning : {acc}")

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")